In [1]:
import cv2
import torch
import numpy as np
from pathlib import Path
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import deque
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from transformers import pipeline
import datetime
import os
from langchain.llms import Ollama

# OBJECT DETECTION (PERSON)
## YOLOV5

In [2]:
# Suppress the FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [0]  # Only detect persons (class 0 in COCO dataset)
model.conf = 0.5  # Confidence threshold
model.cpu()  # Ensure model runs on CPU

Using cache found in C:\Users\ybadr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-6 Python-3.12.3 torch-2.4.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

# Gender Classification

In [3]:
# Load gender classification model
gender_model = load_model('cctv_gender_classifier.h5')
gender_classes = ['man', 'woman']

# Initialize DeepSort
tracker = DeepSort(max_age=10, 
                   n_init=3,
                   nms_max_overlap=1.0,
                   max_cosine_distance=0.3,
                   nn_budget=None,
                   override_track_class=None,
                   embedder="mobilenet",
                   half=True,
                   bgr=True,
                   embedder_gpu=False,
                   embedder_model_name=None,
                   embedder_wts=None,
                   polygon=False,
                   today=None)

In [4]:
# Load the action recognition pipeline
action_pipe = pipeline("image-classification", model="rvv-karma/Human-Action-Recognition-VIT-Base-patch16-224", framework="tf")

def preprocess_body(body_crop, target_size=(126, 126)):
    try:
        body_crop = cv2.resize(body_crop, target_size)
        body_crop = body_crop.astype("float32") / 255.0
        body_crop = img_to_array(body_crop)
        body_crop = np.expand_dims(body_crop, axis=0)
        return body_crop
    except Exception as e:
        print(f"Error in preprocess_body: {e}")
        return None


All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [8]:
# Initialize variables
total_persons = 0
gender_counts = {'man': 0, 'woman': 0}
frame_skip = 2
processing_times = []
fighting_count = 0
lone_woman_flag = False
surrounded_woman_flag = False

# New variables for report generation
sos_events = []
alerts = []
warnings = []
gender_counts_over_time = []

# CCTV Surveillance

In [9]:
# Open video file
video = cv2.VideoCapture(r"C:\Users\ybadr\OneDrive\Desktop\SIH VIDS\Horrifying CCTV footage shows girl abducted, molested in Bengaluru.mp4")
frame_count = 0

# Variables for SOS recording
sos_recording = False
output_video = None
output_filename = ""

# Create a folder to store SOS recordings
sos_folder = "SOS_Recordings"
if not os.path.exists(sos_folder):
    os.makedirs(sos_folder)

def is_night(hour):
    return hour < 6 or hour >= 18

def print_alert(message):
    current_time = datetime.datetime.now()
    alert_info = {
        "message": message,
        "date": current_time.strftime('%Y-%m-%d'),
        "time": current_time.strftime('%I:%M:%S %p'),
        "location": "XYZ"
    }
    alerts.append(alert_info)
    print(f"Alert: {message}")
    print(f"Date: {alert_info['date']}")
    print(f"Time: {alert_info['time']}")
    print(f"Location: {alert_info['location']}")
    print("-------------------")

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
    
    # Resize frame for faster processing
    frame = cv2.resize(frame, (640, 480))
    
    # Get current date and time
    current_time = datetime.datetime.now()
    time_str = current_time.strftime("%Y-%m-%d %I:%M:%S %p")
    cv2.putText(frame, time_str, (320, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(frame, time_str, (320, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Determine if it's day or night
    is_night_time = is_night(current_time.hour)
    day_night_str = "Night" if is_night_time else "Day"
    cv2.putText(frame, day_night_str, (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(frame, day_night_str, (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # YOLOv5 detection
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()
    
    # Prepare detections for DeepSort
    deepsort_detections = []
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        deepsort_detections.append(([x1, y1, x2 - x1, y2 - y1], conf, int(cls)))
    
    # Update DeepSort
    tracks = tracker.update_tracks(deepsort_detections, frame=frame)
    
    # Reset gender counts for each frame
    gender_counts = {'man': 0, 'woman': 0}
    
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        
        # Perform gender classification if not already done
        if not hasattr(track, 'gender'):
            body_crop = frame[y1:y2, x1:x2]
            preprocessed_body = preprocess_body(body_crop)
            if preprocessed_body is not None:
                try:
                    gender_conf = gender_model.predict(preprocessed_body)[0]
                    gender_idx = np.argmax(gender_conf)
                    
                    if gender_conf[gender_idx] > 0.7:  # Confidence threshold
                        gender_label = gender_classes[gender_idx]
                        track.gender = gender_label
                        track.gender_confidence = gender_conf[gender_idx]
                        total_persons += 1
                except Exception as e:
                    print(f"Error in gender classification: {e}")
        
        # Count genders in current frame
        if hasattr(track, 'gender'):
            gender_counts[track.gender] += 1
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"ID {track_id}"
        if hasattr(track, 'gender'):
            label += f" ({track.gender})"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Perform action recognition
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)
    action_result = action_pipe(pil_image)
    
    # Check for fighting action
    sos_detected = False
    for item in action_result:
        if item['label'] == 'Fighting' and item['score'] > 0.97:
            sos_detected = True
            sos_event = {
                "type": "Fighting",
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
                "location": "XYZ",
                "men_count": gender_counts['man'],
                "women_count": gender_counts['woman']
            }
            sos_events.append(sos_event)
            print_alert("SOS! Fighting detected. Action must be taken.")
            cv2.putText(frame, "ALERT! SOS!!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            lone_woman_flag = False
            surrounded_woman_flag = False
            
            # Start recording if not already
            if not sos_recording:
                sos_recording = True
                output_filename = os.path.join(sos_folder, f"XYZ_{current_time.strftime('%Y%m%d_%H%M%S')}.mp4")
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                output_video = cv2.VideoWriter(output_filename, fourcc, 30.0, (640, 480))
            
            break

    # Update flags based on current frame
    if gender_counts['woman'] == 1 and gender_counts['man'] == 0:
        lone_woman_flag = True
    elif gender_counts['woman'] == 1 and gender_counts['man'] >= 2:
        surrounded_woman_flag = True

    # Display warnings based on flags
    if not sos_detected:
        if lone_woman_flag:
            warning_msg = "Warning: Lone woman at night" if is_night_time else "Notice: Lone woman at daytime"
            cv2.putText(frame, warning_msg, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            warnings.append({
                "message": warning_msg,
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
                "location": "XYZ"
            })
            print_alert(f"{warning_msg}. There is potential threat.")
        elif surrounded_woman_flag:
            warning_msg = "Warning: Woman surrounded by men"
            cv2.putText(frame, warning_msg, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            warnings.append({
                "message": warning_msg,
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
                "location": "XYZ"
            })
            print_alert(f"{warning_msg}. There is potential threat.")

    # Record frame if SOS is active
    if sos_recording and output_video is not None:
        output_video.write(frame)

    # Display stats on the frame
    cv2.putText(frame, f"Men: {gender_counts['man']}, Women: {gender_counts['woman']}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Store gender counts over time
    gender_counts_over_time.append({
        "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
        "men_count": gender_counts['man'],
        "women_count": gender_counts['woman']
    })

    # Display the video frame
    cv2.imshow('Full Body Gender Detection and Action Recognition', frame)
 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:21 AM
Location: XYZ
-------------------
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:21 AM
Location: XYZ
-------------------
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:22 AM
Location: XYZ
-------------------
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:22 AM
Location: XYZ
-------------------
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:23 AM
Location: XYZ
-------------------
Alert: Warning: Woman surrounded by men. There is potential threat.
Date: 2024-09-23
Time: 12:52:23 AM
Location: XYZ
-------

In [7]:
# Release resources
video.release()
if output_video is not None:
    output_video.release()
cv2.destroyAllWindows()
if sos_recording:
    print(f"SOS video saved as: {output_filename}")

In [19]:
# Print summary of collected data
print(f"Total SOS events: {len(sos_events)}")
print(f"Total alerts: {len(alerts)}")
print(f"Total warnings: {len(warnings)}")
print(f"Gender counts recorded over is {total_persons-5} time ")

Total SOS events: 11
Total alerts: 83
Total warnings: 72
Gender counts recorded over is 3 time 


In [20]:
import json
from langchain.llms import Ollama

# REPORT GENERATION

In [21]:
# Function to generate a report using the Gemma 2B LLM
def generate_report(sos_events, alerts, warnings, gender_counts_over_time):
    # Prepare the input data for the LLM
    input_data = {
        "sos_events": sos_events,
        "alerts": alerts,
        "warnings": warnings,
        "gender_counts": gender_counts_over_time
    }

    # Convert the input data to a JSON string
    input_json = json.dumps(input_data, indent=2)

    # Prepare the prompt for the LLM
    prompt = f"""As an AI assistant, analyze the following CCTV surveillance data and generate a comprehensive report for authorities. The data includes SOS events, alerts, warnings, and gender counts over time.

DATA:
{input_json}

Based on this data, please generate a detailed report that includes:
1. An executive summary of the surveillance period
2. A breakdown of SOS events, including their nature, timing, and location
3. A summary of alerts and warnings issued during the surveillance period
4. Analysis of gender distribution and any patterns or anomalies observed

Please format the report in a clear, professional manner suitable for law enforcement and city officials."""

    # Initialize the Gemma 2B LLM
    llm = Ollama(model="gemma2:2b")

    # Generate the report
    try:
        report = llm.predict(prompt)
        return report
    except Exception as e:
        print(f"Error generating report: {e}")
        return None

# Generate the report
report = generate_report(sos_events, alerts, warnings, gender_counts_over_time)

# Print or save the report
if report:
    print("Generated Report:")
    print(report)
    
    # Optionally, save the report to a file
    with open("surveillance_report.txt", "w") as f:
        f.write(report)
    print("Report saved to surveillance_report.txt")
else:
    print("Failed to generate report.")

Generated Report:
## Surveillance Report: September 7th, 2024

**Executive Summary:** This report summarizes the events and data collected between the specified start and end times on September 7th, 2024. The surveillance period involved analyzing a wide range of activity within the city's monitored areas. The data reveals ongoing activity with high levels of observed incidents including SOS calls for assistance and potential security concerns.  We are also observing notable patterns in gender distribution related to SOS events. Further investigation is recommended to understand these trends, particularly concerning vulnerable populations.

**1. SOS Event Breakdown:**
* **Total SOS Calls:** The total number of SOS calls received during the surveillance period was [Number]. 
    * **Nature of Events:** The majority of SOS events involved [brief description of common nature e.g., "lost persons", "medical emergency", "home intrusion"].  A smaller portion of events were [mention other even